In [1]:
import numpy as np
import pandas as pd

import datetime

import lightgbm as lgb
from sklearn.metrics import mean_absolute_error
import re

from data_preprocess import DataTransformer
random_state = 12345

ModuleNotFoundError: No module named 'data_preprocess'

In [ ]:


def features_interval(features, target, date1, date2):
    """
    Функция для выделения временных интервалов из таблиц признаков и целей
    на этом этапе отбрасываем колонку 'date'
    """
    features_interval = features[ (features['date']>=date1) & (features['date']<date2) ]
    target_interval = target[features_interval.index]
    features_interval = features_interval.drop('date', axis=1)
    
    return features_interval, target_interval

In [ ]:
def mae_day(y_true, y_pred):
    """
    Функция для вычисления метрики mae по дням из почасовых массивов данных
    """
    y_true_copy = pd.DataFrame(y_true).reset_index(drop=True)
    y_true_copy['day'] = y_true_copy.index // 24
    y_true_grouped = y_true_copy.groupby(by='day').sum()   
    y_pred_copy = pd.DataFrame(y_pred).reset_index(drop=True)
    y_pred_copy['day'] = y_pred_copy.index // 24
    y_pred_grouped = y_pred_copy.groupby(by='day').sum()
    
    return mean_absolute_error(y_true_grouped, y_pred_grouped)

In [ ]:
transformer = DataTransformer()

In [ ]:
path = 'data/df_test_2222.csv'
all_ds, test_begin, test_end = transformer.open_file(path)

начало закрытого теста: 2023-04-01 00:00:00     конец закрытого теста: 2023-08-01 00:00:00


In [ ]:
all_ds = transformer.transform(all_ds)

In [ ]:
# Отбираем признаки. Все лишние колонки здесь отбрасываем, кроме 'date', которую уберем позже 
feature_cols = list(all_ds.columns)

# выбрасываем взгляд в прошлое и расшифрованную погоду
drop_list = ['target', 'day_of_year', 'weather_pred', 'weather_fact', 'temp']

# выбрасываем признаки, найденные процедурно в процессе оптимизации
drop_list = drop_list + ['target_lag_48', 'target_lag_168'] 

for name in drop_list:
    feature_cols.remove(name)


In [ ]:
# Формируем набор датасетов для обучения и проверки

features = all_ds[feature_cols]
target = all_ds['target']

# готовим учебный и тестовый датасеты
features_all_train, target_all_train = features_interval(features, target, '2019-01-01', test_end)
features_test, target_test = features_interval(features, target, test_begin, test_end)

In [ ]:

# Здесь обучаем на всех данных, которые были нам предоставлены 
lgbm_model_all_train = lgb.LGBMRegressor(num_leaves=15, learning_rate=0.02, num_iterations=10000, 
                               feature_fraction=0.987, random_state=random_state, objective='regression_l1', n_jobs=-1)
lgbm_model_all_train.fit(features_all_train, target_all_train)


/opt/anaconda3/envs/p311/lib/python3.11/site-packages/lightgbm/engine.py:172: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


[LightGBM] [Warning] feature_fraction is set=0.987, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.987
[LightGBM] [Warning] feature_fraction is set=0.987, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.987
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.002814 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1537
[LightGBM] [Info] Number of data points in the train set: 45798, number of used features: 27
[LightGBM] [Info] Start training from score 462.615997


LGBMRegressor(feature_fraction=0.987, learning_rate=0.02, n_jobs=-1,
              num_iterations=10000, num_leaves=15, objective='regression_l1',
              random_state=12345)

In [ ]:
predict = lgbm_model_all_train.predict(features_test)
predict_result = pd.DataFrame( features.loc[features_test.index, 'date'] )
predict_result['predict'] = predict
predict_result = predict_result.groupby(by='date').sum().reset_index()

[LightGBM] [Warning] feature_fraction is set=0.987, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.987


In [ ]:
predict_result

,date,predict
0,2023-04-01,23524.651189
1,2023-04-02,23965.122216
2,2023-04-03,25070.081765
3,2023-04-04,26062.596670
4,2023-04-05,26149.703818
...,...,...
117,2023-07-27,19380.391736
118,2023-07-28,19513.514258
119,2023-07-29,18923.460236
120,2023-07-30,18580.364042
